# [Tutorial](https://github.com/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/Stock_NeurIPS2018/Stock_NeurIPS2018_1_Data.ipynb) 


# Importando as bibliotecas

In [2]:
## install required packages
#conda install python==3.10
"""!pip install swig
!pip install wrds
!pip install pyportfolioopt
!pip install QuantStats
!pip install finrl
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git
!pip install yfinance
!pip install finrl
!pip install Pillow==9.5.0
!pip install matplotlib==3.7.3
!pip install torch==2.0.0
#!pip install swig"""

## install finrl library
#!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

'!pip install swig\n!pip install wrds\n!pip install pyportfolioopt\n!pip install QuantStats\n!pip install finrl\n!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git\n!pip install yfinance\n!pip install finrl\n!pip install Pillow==9.5.0\n!pip install matplotlib==3.7.3\n!pip install torch==2.0.0\n#!pip install swig'

In [1]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

In [2]:
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

# Importando os dados

In [3]:
# Lista consolidada de códigos de ações
codigo_acoes_bovespa = [
    # Bens Industriais
    'BHIA3', 'RZAT11', 'GRWA11', 'CRAA11', 'ZAMP3', 'HGAG11', 'BBGO11', 'AGRX11', 'PLCA11', 'RURA11',
    'SNAG11', 'GCRA11', 'VCRA11', 'KNCA11', 'NCRA11', 'CPTR11', 'FGAA11', 'EGAF11', 'VGIA11', 'LSAG11',
    'N2ET34', 'M1TA34', 'FOOD11', 'AERI3', 'ICBR3', 'DOTZ3', 'GOLL3', 'VIIA3', 'ARML3', 'MLAS3', 'CBAV3',
    'TTEN3', 'BRBI11', 'NINJ3', 'ATEA3', 'MODL4', 'MODL3', 'VITT3', 'KRSA3',
    # Consumo Cíclico
    'ALOS3', 'VIIA3', 'CEDO4', 'CEDO3', 'NFLX34', 'NIKE34', 'MCDC34', 'HOME34', 'FDMO34', 'CMCS34', 'AMZO34',
    'RDNI3', 'SLED4', 'RSID3', 'MNDL3', 'LEVE3', 'CTKA4', 'CTKA3', 'MYPK3', 'GRND3', 'LCAM3', 'CEAB3', 'VSTE3',
    'CGRA3', 'ESTR4', 'ESTR3', 'DIRR3', 'CTNM4', 'CTNM3', 'ANIM3', 'DIRR3', 'EVEN3', 'AMAR3', 'MOVI3', 'JHSF3',
    'HBOR3', 'PDGR3', 'ARZZ3', 'EZTC3', 'ALPA3', 'RENT3', 'MRVE3', 'MGLU3', 'LREN3', 'COGN3', 'WHRL4', 'WHRL3',
    'TCSA3', 'SBUB34', 'SMLS3', 'SEER3', 'SLED4', 'HOOT4', 'GFSA3', 'YDUQ3', 'CYRE3',
    # Consumo não Cíclico
    'PRVA3', 'WALM34', 'SBUB34', 'PGCO34', 'PEPB34', 'COLG34', 'COCA34', 'AVON34', 'SMTO3', 'MDIA3', 'CAML3',
    'AGRO3', 'BEEF3', 'VIVA3', 'CRFB3', 'PCAR4', 'PCAR3', 'NTCO3', 'MRFG3', 'JBSS3', 'PGCO34', 'BRFS3',
    # Financeiro
    'NDIV11', 'CSUD3', 'INBR31', 'BIDI3', 'BIDI11', 'BIDI4', 'STOC31', 'NUBR33', 'IGTI11', 'IGTI3', 'XPBR31',
    'TRAD3', 'BSLI4', 'BSLI3', 'WFCO34', 'VISA34', 'MSBR34', 'MSCD34', 'JPMC34', 'HONB34', 'GEOO34', 'GSGI34',
    'CTGP34', 'BOAC34', 'MMMC34', 'SCAR3', 'LPSB3', 'BMGB11', 'IGBR3', 'GSHP3', 'PSSA3', 'CARD3', 'BBRK3', 'BRPR3',
    'BRSR6', 'BRSR5', 'BRSR3', 'SANB4', 'SANB3', 'SANB11', 'MULT3', 'ITUB3', 'ITUB4', 'ALSO3', 'BMIN3', 'MERC4',
    'LOGG3', 'ITSA4', 'IRBR3', 'PDTC3', 'SYNE3', 'BBDC4', 'BRML3', 'APER3', 'BBSE3', 'BPAN4', 'BBAS3', 'AXPB34',
    # Materiais Básicos
    'LAND3', 'DEXP4', 'DEXP3', 'RANI3', 'FCXO34', 'PMAM3', 'FESA4', 'FESA3', 'EUCA3', 'EUCA4', 'SUZB3', 'KLBN4',
    'DXCO3', 'CSNA3', 'BRKM6', 'BRKM5', 'BRKM3', 'BRAP4', 'ARMT34',
    # Outros
    'RBIV11', 'CPLE11', 'GTLG11', 'PPLA11', 'BTLT39', 'BSHY39', 'BSHV39', 'BIEI39', 'BIYT39', 'BGOV39', 'ALUG11',
    'WRLD11', 'CXAG11', 'ROOF11', 'JGPX11', 'PURB11', 'BIME11', 'JSAF11', 'TELD11', 'MORC11', 'HUSI11', 'CYCR11',
    'EQIR11', 'CACR11', 'RZAG11', 'PORT3', 'GETT11', 'GETT4', 'GETT3', 'BIYE39', 'BSCZ39', 'BUSA39', 'BERU39',
    'BSOX39', 'BFCG39', 'BFXH39', 'BFTA39', 'BKYY39', 'BQQW39', 'BFPI39', 'BFDN39', 'BFDA39', 'BFPX39', 'BCIR39',
    'BFBI39', 'BOEF39', 'BICL39', 'C2OI34', 'S2TO34', 'MILA', 'CSMO', 'YDRO11', 'SPXB11', 'SMAB11', 'W2ST34',
    'S2QS34', 'P2AT34', 'G2DD34', 'D2AS34', 'C2PT34', 'BIVW39', 'BIVE39', 'BCWV39', 'A2VL34', 'A2MC34', 'AFHI11',
    'HSRE11', 'VSEC11', 'AGXY3', 'USTK11', 'AGXY3', 'SOJA3', 'Z2NG34', 'T2TD34', 'T2DH34', 'S2UI34', 'S2QU34',
    'S2NW34', 'S2HO34', 'C2ZR34', 'U2ST34', 'P2EN34', 'M2PW34', 'K2CG34', 'D2KN34', 'C2ON34', 'C2HD34', 'B2YN34',
    'ENMT4', 'ENMT3', 'VBBR3', 'RAIZ4', 'RECV3', 'SLBG34', 'HALI34', 'COPH34', 'PRIO3', 'OSXB3', 'DMMO11', 'RPMG3',
    'UGPA3', 'PETR4', 'EXXO34', 'ENAT3',
    # Saúde
    'ONCO3', 'VVEO3', 'PARD3', 'BIOM3', 'BALM4', 'PFIZ34', 'MRCK34', 'GBIO33', 'PNVL3', 'AALR3', 'ODPV3', 'RADL3',
    'QUAL3', 'OFSA3', 'JNJB34', 'HYPE3', 'FLRY3',
    # Tecnologia da Informação
    'CLSA3', 'LVTC3', 'G2DI33', 'IFCM3', 'GOGL35', 'LWSA3', 'TOTS3', 'XRXB34', 'QCOM34', 'ORCL34', 'MSFT34', 'IBMB34',
    'ITLC34', 'HPQB34', 'EBAY34', 'CSCO34', 'ATTB34', 'AAPL34', 'LINX3', 'POSI3',
    # Telecomunicações
    'BRIT3', 'FIQE3', 'DESK3', 'VERZ34', 'OIBR4', 'TIMS3', 'VIVT3', 'TELB4', 'ATTB34',
    # Utilidade Pública
    'AURE3', 'MEGA3', 'CEPE6', 'CEPE5', 'CEPE3', 'CEED3', 'CEED4', 'EEEL4', 'EEEL3', 'CASN4', 'CASN3', 'CEGR3',
    'CEBR3', 'CEBR6', 'CEBR5', 'RNEW11', 'COCE6', 'COCE5', 'COCE3', 'CLSC4', 'CLSC3', 'ALUP4', 'ALUP3', 'SAPR11',
    'SAPR4', 'SAPR3', 'CPRE3', 'CPFE3', 'CGAS3', 'AESB3', 'NEOE3', 'TRPL4', 'EGIE3', 'TAEE4', 'TAEE3', 'TAEE11',
    'SBSP3', 'RNEW11', 'GEPA4', 'CESP6', 'CESP5', 'CESP3', 'CMIG4', 'CMIG3', 'AFLT3'
]
# Adiciona '.SA' a cada código de ação
codigo_acoes_sa = [codigo + '.SA' for codigo in codigo_acoes_bovespa]

In [4]:
# Códigos de ações do índice Ibovespa
codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',
    'BRDT3', 'BRFS3', 'BRKM5', 'BRML3', 'BTOW3', 'CCRO3', 'CIEL3', 'CMIG4', 'COGN3', 'CPFE3',
    'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3',
    'ENBR3', 'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4', 'GNDI3', 'GOAU4', 'GOLL4', 'HAPV3',
    'HGTX3', 'HYPE3', 'IGTA3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LAME4', 'LREN3',
    'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PRIO3', 'QUAL3',
    'RADL3', 'RAIL3', 'RENT3', 'SANB11', 'SBSP3', 'SULA11', 'SUZB3', 'TAEE11', 'TIMS3', 'TOTS3',
    'UGPA3', 'USIM5', 'VALE3', 'VIVT4', 'VVAR3', 'WEGE3', 'YDUQ3'
]
codigos_ibovespa = [codigo + '.SA' for codigo in codigos_ibovespa]

In [5]:
ibovespa_yahoo = yf.download(tickers = codigos_ibovespa, start='2020-01-01', end='2020-01-31')

[*********************100%%**********************]  77 of 77 completed


11 Failed downloads:
['BTOW3.SA', 'ENBR3.SA', 'BRDT3.SA', 'VIVT4.SA', 'GNDI3.SA', 'VVAR3.SA', 'IGTA3.SA', 'LAME4.SA', 'BRML3.SA', 'HGTX3.SA', 'SULA11.SA']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [7]:
ibovespa_yahoo

Adj Close                                              \
                      ABEV3.SA   AZUL4.SA   B3SA3.SA   BBAS3.SA   BBDC3.SA   
Date                                                                         
2020-01-02 00:00:00  17.020620  58.799999  13.083994  41.642776  22.546173   
2020-01-03 00:00:00  16.781267  56.759998  12.706933  41.573116  22.194355   
2020-01-06 00:00:00  16.861052  55.000000  12.585115  41.023552  22.067059   
2020-01-07 00:00:00  16.896511  56.820000  12.991179  40.713943  21.691528   
2020-01-08 00:00:00  16.816727  56.919998  12.956374  40.342407  21.360554   
2020-01-09 00:00:00  16.683754  57.279999  12.950574  39.521946  21.150509   
2020-01-10 00:00:00  16.745808  57.500000  12.849057  38.593098  20.781351   
2020-01-13 00:00:00  17.029486  58.299999  12.863561  39.026558  21.035942   
2020-01-14 00:00:00  16.931971  59.189999  12.950574  38.817574  21.080494   
2020-01-15 00:00:00  16.630564  59.299999  12.602516  38.105461  20.596760   
2020-01-16 00:00:00  16.577375  59.439999  12.747540  38.275738  20.934107   
2020-01-17 00:00:00  16.577375  61.349998  13.150705  38.794353  21.373283   
2020-01-20 00:00:00  16.666025  61.770000  13.197113  38.546654  21.042311   
2020-01-21 00:00:00  16.612835  60.049999  12.965077  37.432060  20.310350   
2020-01-22 00:00:00  16.887646  59.599998  13.777204  37.726189  20.482201   
2020-01-23 00:00:00  16.533052  61.200001  14.125261  39.847038  21.035942   
2020-01-24 00:00:00  16.834455  59.459999  14.209374  39.320683  20.921375   
2020-01-27 00:00:00  16.444399  57.480000  13.820713  38.933666  20.367634   
2020-01-28 00:00:00  16.666025  62.410000  14.235479  39.251022  20.418552   
2020-01-29 00:00:00  16.222776  59.869999  14.096257  38.283497  19.985733   
2020-01-30 00:00:00  16.258240  59.709999  14.154265  38.546654  20.176683   

                                                                            \
                      BBDC4.SA   BBSE3.SA   BEEF3.SA  BPAC11.SA   BRAP4.SA   
Date                                                                         
2020-01-02 00:00:00  24.020878  27.180849   9.786340  17.499079  18.852846   
2020-01-03 00:00:00  24.033037  26.881214  10.043260  17.549860  19.120544   
2020-01-06 00:00:00  23.603764  27.323530  10.035476  17.524467  19.209780   
2020-01-07 00:00:00  23.193703  27.337795  10.494817  17.594383  19.294052   
2020-01-08 00:00:00  22.834913  27.323530  10.611601  17.438457  19.289093   
2020-01-09 00:00:00  22.469706  27.245052  10.448106  17.557150  19.041225   
2020-01-10 00:00:00  22.059647  26.921337  10.401392  17.803843  18.981737   
2020-01-13 00:00:00  22.296715  26.721642  11.335649  17.850388  19.779873   
2020-01-14 00:00:00  22.271076  27.039665  11.250009  17.510603  19.968250   
2020-01-15 00:00:00  21.880247  27.069248  11.401547  17.361654  20.017826   
2020-01-16 00:00:00  22.194202  26.677265  11.868693  17.140564  19.948423   
2020-01-17 00:00:00  22.713173  26.729036  11.757845  17.834095  20.806047   
2020-01-20 00:00:00  22.271076  26.743828  11.773681  17.803843  21.014256   
2020-01-21 00:00:00  21.527853  26.625492  12.050802  17.892275  20.617666   
2020-01-22 00:00:00  21.604748  26.766018  11.124426  17.668858  20.944853   
2020-01-23 00:00:00  22.174973  26.802998  11.298616  18.246029  20.652369   
2020-01-24 00:00:00  22.008396  26.551535  10.372241  18.190172  20.245865   
2020-01-27 00:00:00  21.502235  25.885895   9.525043  17.459402  18.966866   
2020-01-28 00:00:00  21.527853  25.893291   9.620056  17.505947  18.961908   
2020-01-29 00:00:00  21.220322  26.344446   9.445866  17.550167  18.604979   
2020-01-30 00:00:00  21.431749  26.337053   9.042060  17.191765  18.674379   

                     ...    Volume                                         \
                     ... TAEE11.SA  TIMS3.SA  TOTS3.SA UGPA3.SA  USIM5.SA   
Date                 ...                                                    
2020-01-02 00:00:00  ...   1270600   8256700

In [8]:
df_ibovespa = YahooDownloader(start_date ='2009-01-01',
                end_date = '2022-01-31',
                ticker_list =codigos_ibovespa).fetch_data()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['BRDT3.SA']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['BRML3.SA']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed

...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['VIVT4.SA']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['VVAR3.SA']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Shape of DataFrame:  (192064, 8)


In [9]:
df_ibovespa['tic'].value_counts()

tic
ABEV3.SA    3243
RADL3.SA    3243
ITSA4.SA    3243
ITUB4.SA    3243
JBSS3.SA    3243
            ... 
AZUL4.SA    1195
CRFB3.SA    1126
IRBR3.SA    1119
HAPV3.SA     930
NTCO3.SA     522
Name: count, Length: 66, dtype: int64

In [10]:
numero_todas_datas = df_ibovespa['tic'].value_counts().iloc[0]
numero_todas_datas

3243

In [11]:
numero_todas_datas

3243

In [12]:
##Filtrando os ativos com muitos todos os dados no historico
selected_tickers = df_ibovespa['tic'].value_counts()[df_ibovespa['tic'].value_counts() >= numero_todas_datas ].index

# Use os tickers para filtrar o DataFrame
filtered_df = df_ibovespa[df_ibovespa['tic'].isin(selected_tickers)]

In [13]:
print('Com o filtro sobram o total de:',filtered_df['tic'].value_counts().index.nunique(),'ações')

Com o filtro sobram o total de: 45 ações


In [14]:
filtered_df['tic'].value_counts()

tic
ABEV3.SA     3243
HYPE3.SA     3243
ITUB4.SA     3243
JBSS3.SA     3243
LREN3.SA     3243
MRFG3.SA     3243
MRVE3.SA     3243
MULT3.SA     3243
PCAR3.SA     3243
PETR3.SA     3243
PETR4.SA     3243
RADL3.SA     3243
RENT3.SA     3243
SBSP3.SA     3243
TAEE11.SA    3243
TIMS3.SA     3243
TOTS3.SA     3243
UGPA3.SA     3243
USIM5.SA     3243
VALE3.SA     3243
WEGE3.SA     3243
ITSA4.SA     3243
GOLL4.SA     3243
BBAS3.SA     3243
GOAU4.SA     3243
BBDC3.SA     3243
BBDC4.SA     3243
BEEF3.SA     3243
BRAP4.SA     3243
BRFS3.SA     3243
BRKM5.SA     3243
CCRO3.SA     3243
CMIG4.SA     3243
CPFE3.SA     3243
CSAN3.SA     3243
CSNA3.SA     3243
CYRE3.SA     3243
EGIE3.SA     3243
ELET3.SA     3243
ELET6.SA     3243
EMBR3.SA     3243
EQTL3.SA     3243
EZTC3.SA     3243
GGBR4.SA     3243
YDUQ3.SA     3243
Name: count, dtype: int64

In [15]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145935 entries, 0 to 192063
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    145935 non-null  object 
 1   open    145935 non-null  float64
 2   high    145935 non-null  float64
 3   low     145935 non-null  float64
 4   close   145935 non-null  float64
 5   volume  145935 non-null  int64  
 6   tic     145935 non-null  object 
 7   day     145935 non-null  int32  
dtypes: float64(4), int32(1), int64(1), object(2)
memory usage: 9.5+ MB


# Processando os dados

In [16]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(filtered_df)

Successfully added technical indicators
[*********************100%%**********************]  1 of 1 completed
Shape of DataFrame:  (3291, 8)
Successfully added vix
Successfully added turbulence index


In [17]:
processed.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,3.333858,3.556008,3.333858,2.150698,590662,ABEV3.SA,4,0.0,2.216974,2.011928,0.0,66.666667,100.0,2.150698,2.150698,39.189999,0.0
1,2009-01-02,14.680000,15.250000,14.600000,5.934311,2801600,BBAS3.SA,4,0.0,2.216974,2.011928,0.0,66.666667,100.0,5.934311,5.934311,39.189999,0.0
2,2009-01-02,5.856117,6.178203,5.738994,3.445034,616107,BBDC3.SA,4,0.0,2.216974,2.011928,0.0,66.666667,100.0,3.445034,3.445034,39.189999,0.0
3,2009-01-02,14.166245,14.166245,14.166245,7.943635,0,BBDC4.SA,4,0.0,2.216974,2.011928,0.0,66.666667,100.0,7.943635,7.943635,39.189999,0.0
4,2009-01-02,1.858427,2.015754,1.819095,1.486063,230958,BEEF3.SA,4,0.0,2.216974,2.011928,0.0,66.666667,100.0,1.486063,1.486063,39.189999,0.0


In [18]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [19]:
processed_full.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,ABEV3.SA,3.333858,3.556008,3.333858,2.150698,590662.0,4.0,0.0,2.216974,2.011928,0.0,66.666667,100.0,2.150698,2.150698,39.189999,0.0
1,2009-01-02,BBAS3.SA,14.680000,15.250000,14.600000,5.934311,2801600.0,4.0,0.0,2.216974,2.011928,0.0,66.666667,100.0,5.934311,5.934311,39.189999,0.0
2,2009-01-02,BBDC3.SA,5.856117,6.178203,5.738994,3.445034,616107.0,4.0,0.0,2.216974,2.011928,0.0,66.666667,100.0,3.445034,3.445034,39.189999,0.0
3,2009-01-02,BBDC4.SA,14.166245,14.166245,14.166245,7.943635,0.0,4.0,0.0,2.216974,2.011928,0.0,66.666667,100.0,7.943635,7.943635,39.189999,0.0
4,2009-01-02,BEEF3.SA,1.858427,2.015754,1.819095,1.486063,230958.0,4.0,0.0,2.216974,2.011928,0.0,66.666667,100.0,1.486063,1.486063,39.189999,0.0


# Salvando os dados

In [20]:
porcentagem_treino = 0.8

In [21]:
index_data_inical = int(processed_full['date'].unique().shape[0] *porcentagem_treino)
index_data_inical

2529

In [22]:
treino_inicio = processed['date'].min()
treino_final =  processed_full['date'].unique()[:index_data_inical].max()
teste_inicio = processed_full['date'].unique()[index_data_inical:].min()
teste_final = processed_full['date'].unique()[index_data_inical:].max()

In [23]:
train = data_split(processed_full, treino_inicio,treino_final)
trade = data_split(processed_full, teste_inicio,teste_final)

In [24]:
print('Porcentagem de',porcentagem_treino, 'para o treinamento e', round(1-porcentagem_treino,1),'para o teste')
print('Total de',len(train),'de dados para o treinamento e', len(trade),'para o teste')
print('Data do incio do treino:',treino_inicio)
print('Data do termino do treino:',treino_final)
print('Data do incio do teste:',teste_inicio)
print('Data do termino do teste:',teste_final)

Porcentagem de 0.8 para o treinamento e 0.2 para o teste
Total de 113760 de dados para o treinamento e 28440 para o teste
Data do incio do treino: 2009-01-02
Data do termino do treino: 2019-06-18
Data do incio do teste: 2019-06-19
Data do termino do teste: 2022-01-27


In [25]:
train.to_csv('train_data_ibovespa.csv')
trade.to_csv('trade_data_ibovespa.csv')

In [26]:
train

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,ABEV3.SA,3.333858,3.556008,3.333858,2.150698,590662.0,4.0,0.000000,2.216974,2.011928,0.000000,66.666667,100.000000,2.150698,2.150698,39.189999,0.000000
0,2009-01-02,BBAS3.SA,14.680000,15.250000,14.600000,5.934311,2801600.0,4.0,0.000000,2.216974,2.011928,0.000000,66.666667,100.000000,5.934311,5.934311,39.189999,0.000000
0,2009-01-02,BBDC3.SA,5.856117,6.178203,5.738994,3.445034,616107.0,4.0,0.000000,2.216974,2.011928,0.000000,66.666667,100.000000,3.445034,3.445034,39.189999,0.000000
0,2009-01-02,BBDC4.SA,14.166245,14.166245,14.166245,7.943635,0.0,4.0,0.000000,2.216974,2.011928,0.000000,66.666667,100.000000,7.943635,7.943635,39.189999,0.000000
0,2009-01-02,BEEF3.SA,1.858427,2.015754,1.819095,1.486063,230958.0,4.0,0.000000,2.216974,2.011928,0.000000,66.666667,100.000000,1.486063,1.486063,39.189999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2527,2019-06-17,UGPA3.SA,21.500000,21.760000,21.200001,19.576727,4782300.0,0.0,0.037132,19.712595,17.259325,49.534814,109.201347,16.717154,18.651063,19.546139,15.350000,29.502932
2527,2019-06-17,USIM5.SA,8.640000,8.740000,8.580000,7.111204,9301100.0,0.0,0.053284,7.430484,6.509956,48.593497,79.133821,1.498121,6.903104,7.265732,15.350000,29.502932
2527,2019-06-17,VALE3.SA,51.000000,51.330002,50.200001,34.242954,20551200.0,0.0,0.309982,35.595954,31.842895,51.304006,100.158034,14.143575,33.405986,34.137912,15.350000,29.502932
2527,2019-06-17,WEGE3.SA,9.975000,10.265000,9.910000,9.559977,6131400.0,0.0,0.175099,9.511833,8.490124,62.708422,186.571970,45.431908,8.863151,8.786260,15.350000,29.502932


In [27]:
trade

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2019-06-19,ABEV3.SA,18.049999,18.360001,17.980000,15.721224,23446200.0,2.0,0.152904,15.663305,14.661381,57.200970,167.722872,23.650660,14.982096,15.036227,14.330000,24.913164
0,2019-06-19,BBAS3.SA,51.500000,52.500000,51.110001,39.596638,7964300.0,2.0,0.529207,40.401258,37.154286,56.015006,66.083933,2.784967,37.781369,37.048178,14.330000,24.913164
0,2019-06-19,BBDC3.SA,24.350113,24.748308,23.944403,19.880020,3485755.0,2.0,0.282393,20.480991,18.284890,56.957863,67.945098,12.518559,18.834974,18.733100,14.330000,24.913164
0,2019-06-19,BBDC4.SA,27.227648,27.723516,26.867018,22.328823,20888447.0,2.0,0.209542,23.129236,20.960027,54.459579,52.308731,9.690325,21.571333,21.516488,14.330000,24.913164
0,2019-06-19,BEEF3.SA,7.866358,7.954854,7.797527,6.243949,6414658.0,2.0,0.037579,6.432561,6.092708,53.460961,-20.314667,15.627085,6.255368,6.108482,14.330000,24.913164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,2022-01-26,UGPA3.SA,14.130000,14.830000,14.020000,13.419630,12434400.0,2.0,-0.083954,13.696630,11.627995,51.625196,45.199194,18.080459,13.047697,13.012193,31.959999,38.328956
631,2022-01-26,USIM5.SA,16.330000,16.709999,15.770000,14.787024,35743200.0,2.0,0.403402,15.820255,12.604494,55.738793,75.502264,10.741102,14.029746,13.035091,31.959999,38.328956
631,2022-01-26,VALE3.SA,85.900002,86.559998,83.820000,72.341820,25806800.0,2.0,1.819475,77.060980,65.054238,56.169662,73.014175,17.610762,69.928430,64.896969,31.959999,38.328956
631,2022-01-26,WEGE3.SA,30.450001,31.340000,30.260000,29.777555,12183300.0,2.0,-0.770862,31.807229,27.337156,42.506854,-37.407664,12.828281,30.782094,32.411607,31.959999,38.328956
